## 获取数据

In [55]:
import json
import datetime 
fid = open("D:\\学术\\work\比赛\\2018.2.21.3nd北邮大数据竞赛\\data\\train.json")
data_raw = json.load(fid)
fid.close()
start_time = datetime.datetime.strptime("2017-10-17", "%Y-%m-%d")
feats_raw = []
labs = []
for data in data_raw:
    data_time = datetime.datetime.strptime(data["date"], "%Y-%m-%d:%H")
    if data_time > start_time:
        feats_raw.append([data["holiday"], data["week"], data["place"], data_time.hour])
        labs.append(data["people"])

In [58]:
from sklearn import preprocessing
ohcoder = preprocessing.OneHotEncoder()
ohcoder.fit(feats_raw)
print(ohcoder.n_values_) # 每个特征对应的最大位数

ImportError: DLL load failed: 找不到指定的模块。

In [56]:
print(ohcoder.transform(feats_raw).toarray())

[[0, 2, 1, 1],
 [0, 2, 2, 1],
 [0, 2, 3, 1],
 [0, 2, 4, 1],
 [0, 2, 5, 1],
 [0, 2, 6, 1],
 [0, 2, 7, 1],
 [0, 2, 8, 1],
 [0, 2, 9, 1],
 [0, 2, 10, 1],
 [0, 2, 11, 1],
 [0, 2, 12, 1],
 [0, 2, 13, 1],
 [0, 2, 14, 1],
 [0, 2, 15, 1],
 [0, 2, 16, 1],
 [0, 2, 17, 1],
 [0, 2, 18, 1],
 [0, 2, 19, 1],
 [0, 2, 20, 1],
 [0, 2, 21, 1],
 [0, 2, 22, 1],
 [0, 2, 23, 1],
 [0, 2, 24, 1],
 [0, 2, 25, 1],
 [0, 2, 26, 1],
 [0, 2, 27, 1],
 [0, 2, 28, 1],
 [0, 2, 29, 1],
 [0, 2, 30, 1],
 [0, 2, 31, 1],
 [0, 2, 32, 1],
 [0, 2, 33, 1],
 [0, 2, 1, 2],
 [0, 2, 2, 2],
 [0, 2, 3, 2],
 [0, 2, 4, 2],
 [0, 2, 5, 2],
 [0, 2, 6, 2],
 [0, 2, 7, 2],
 [0, 2, 8, 2],
 [0, 2, 9, 2],
 [0, 2, 10, 2],
 [0, 2, 11, 2],
 [0, 2, 12, 2],
 [0, 2, 13, 2],
 [0, 2, 14, 2],
 [0, 2, 15, 2],
 [0, 2, 16, 2],
 [0, 2, 17, 2],
 [0, 2, 18, 2],
 [0, 2, 19, 2],
 [0, 2, 20, 2],
 [0, 2, 21, 2],
 [0, 2, 22, 2],
 [0, 2, 23, 2],
 [0, 2, 24, 2],
 [0, 2, 25, 2],
 [0, 2, 26, 2],
 [0, 2, 27, 2],
 [0, 2, 28, 2],
 [0, 2, 29, 2],
 [0, 2, 30, 2],
 [0, 2, 31

In [16]:
data[1].keys()

dict_keys(['week', 'holiday', 'place', 'people', 'date'])

## 划分数据

In [ ]:
from sklearn.model_selection import train_test_split
train_feats, val_feats, train_labs, val_labs = train_test_split(feats, labs, test_size = 0.2, random_state = 0)

## 测试方法

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def fit_and_perform(model, train_feats, train_labs, val_feats, val_labs):
    model.fit(train_feats, train_labs)
    train_score = model.score(train_feats, train_labs)
    train_res = model.predict(train_feats)
    val_score = model.score(val_feats, val_labs)
    val_res = model.predict(val_feats)
    plt.figure(figsize=(20,8))
    plt.subplot(121)
    plt.scatter(train_labs,train_res, color = 'r', s = 20)
    plt.plot([0, 60],[0, 60])
    plt.title('score: %f'%train_score)
    plt.subplot(122)
    plt.scatter(val_labs, val_res, color = 'r', s = 20)
    plt.plot([0, 60],[0, 60])
    plt.title('score: %f'%val_score)
    plt.show()
    print("RMSE:",np.mean((val_res-val_labs)**2)**0.5)

In [ ]:
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
# model = tree.DecisionTreeRegressor() # 决策树
# model = linear_model.LinearRegression() #线性回归
# model = svm.SVR() # SVM
# model = neighbors.KNeighborsRegressor() # KNN
# model = ensemble.RandomForestRegressor(n_estimators=20) # 使用20个决策树的rf
# model = ensemble.AdaBoostRegressor(n_estimators=50) # 使用50个决策树的adaboost
# model = ensemble.GradientBoostingRegressor(n_estimators=100) # 这里使用100个决策树的GBRT
# model_BaggingRegressor = ensemble.BaggingRegressor() #Bagging
# model = ExtraTreeRegressor() # 极端随机树
fit_and_perform(model, train_feats, train_labs, val_feats, val_labs)

In [ ]:
import xgboost as xgb
param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'reg:linear' }
param['nthread'] = 4
plst = param.items()
# plst += [('eval_metric', 'auc')] # Multiple evals can be handled in this way
# plst += [('eval_metric', 'ams@0')]

dtrain = xgb.DMatrix(train_feats, label=train_labs)
num_round = 10
model = xgb.train(plst, dtrain, num_round)
dtest = xgb.DMatrix(train_feats)
train_res = model.predict(dtest)
dtest = xgb.DMatrix(val_feats)
val_res = model.predict(dtest)
plt.figure(figsize=(20,8))
plt.subplot(121)
plt.scatter(train_labs,train_res, color = 'r', s = 20)
plt.plot([0, 60],[0, 60])
plt.subplot(122)
plt.scatter(val_labs, val_res, color = 'r', s = 20)
plt.plot([0, 60],[0, 60])
plt.show()
print("RMSE:",np.mean((val_res-val_labs)**2)**0.5)

## 生成提交数据

In [ ]:
file_name = "D:\\学术\\work\比赛\\2018.2.21.3nd北邮大数据竞赛\\result\\result_2018-05-22.csv"
for i in range(1,34):
    for h in range(24):
        

## trick

In [8]:
file_name = "D:\\学术\\work\比赛\\2018.2.21.3nd北邮大数据竞赛\\result\\result_2018-05-26.csv"
R0 = 1045.2566
R1 = 214.1222

fid = open(file_name,"r")
line = fid.readlines()
fid.close()

C2 = 0
n = 9
for l in line:
    C2 += float(l.strip("\n").split(",")[-1])**2
    n += 1
k = n*(R0**2 - R1**2)/(2*C2) + 0.5

file_name_plus1 = file_name.split(".")
file_name_plus1[-2] = file_name_plus1[-2] + "+1"

fid = open(".".join(file_name_plus1),'w',encoding='utf8')
for l in line:
    tmp = l.strip("\n").split(",")[:-1]
    tmp.append(str(k*float(l.strip("\n").split(",")[-1]))+"\n")
    fid.write(",".join(tmp))
fid.close()

In [9]:
k

1.3827510140812769